<a href="https://colab.research.google.com/github/ArunKoundinya/DeepLearning/blob/main/posts/deep-learning-project-msis/AmazonReviews_Part8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will taking the previous model of basic Attention and then following the below steps

*   Increasing Model Complexity
*   Regularizing the Model
*   Hypertuning the Model
*   Testing the Model on 10K dataset
*   Finally testing the model on 1 Lakh (0.1 Million) dataset


## Table of Contents
- [1 - Packages](#1)
- [2 - Loading the Dataset](#2)
- [3 - Pre-Processing the Data](#3)
- [4 - Model-1 Attention Model](#4)
- [5 - Model-2 Increasing Layers and Units](#5)
- [6 - Model-3 Adding Regularization](#6)
- [7 - Model-4 Hypertuning the model](#7)
- [8 - Model-5 Testing Model on 10K dataset](#8)
- [9 - Model-6 Testing Model on 1 Lac dataset](#9)

<a name='1'></a>
## 1 - Loading the Packages

In [ ]:
!pip install pandarallel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16673 sha256=be7381230ffce7eef1c746056f9539342fe9d214e2273eb451de3ba03a6a86ff
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN, GRU, Bidirectional,Input,Dropout,Flatten,Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.models import Model

from sklearn.metrics import accuracy_score, classification_report

from pandarallel import pandarallel

<a name='2'></a>
## 2 - Loading the Data

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/MSIS/IntroductiontoDeepLearning/Project/')

testdata = pd.read_csv('test_data_sample_complete.csv')
traindata = pd.read_csv('train_data_sample_complete.csv')


Mounted at /content/drive


In [ ]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<a name='3'></a>
## 3 - Pre-Processing the Data

In [ ]:
train_data = traindata.sample(n=10000, random_state=42)
test_data = testdata.sample(n=1000, random_state=42)

train_data['class_index'] = train_data['class_index'].map({1:0, 2:1})
test_data['class_index'] = test_data['class_index'].map({1:0, 2:1})

train_data['review_combined_lemma'] = train_data['review_combined_lemma'].fillna('')
test_data['review_combined_lemma'] = test_data['review_combined_lemma'].fillna('')

X_train = train_data.review_combined_lemma
y_train = np.array(train_data.class_index)

X = test_data.review_combined_lemma
y = np.array(test_data.class_index)

tokenizer = Tokenizer(oov_token="<UNK>",)
tokenizer.fit_on_texts(X_train)

tokenizer.word_index['<PAD>'] = 0

X_sequences_train = tokenizer.texts_to_sequences(X_train)

X_sequences = tokenizer.texts_to_sequences(X)

X_train = pad_sequences(X_sequences_train, maxlen=100)
X = pad_sequences(X_sequences, maxlen=100)

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

(10000, 100)
(500, 100)
(500, 100)


In [ ]:
vocab_size = len(tokenizer.word_index)

In [ ]:
word2idx = tokenizer.word_index

In [ ]:
def load_embeddings(glove_path):
    embedding_index = {}
    with open(glove_path, encoding="utf8") as glove_file:
        for line in glove_file:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embedding_index[word] = coefs
    return embedding_index

In [ ]:
def create_embedding_matrix(embedding_index, word2idx, vocab_size, embedding_dim):
    mat=np.zeros((vocab_size,embedding_dim))
    for key,value in word2idx.items():
      mat[value]=embedding_index.get(key)
    mat[np.isnan(mat)] = 0
    return mat

In [ ]:
glove_path3 = f"glove.6B/glove.twitter.27B.200d.txt"
embedding_index_Twitter_200d = load_embeddings(glove_path3)

In [ ]:
embedding_matrix_twitter_200d = create_embedding_matrix(embedding_index_Twitter_200d, word2idx, vocab_size, 200)

<a name='4'></a>
## 4 - Model 1 - Attention Model from Previous Notebook.

In [ ]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units, activation="tanh")
        self.V = tf.keras.layers.Dense(1)

    def call(self, features):
        # Compute attention scores
        score = self.W1(features)

        # Apply softmax activation to obtain attention weights
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # Compute context vector as the weighted sum of features
        context_vector = attention_weights * features

        return context_vector


In [ ]:
inputs = Input(shape=(100,))

embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(4, activation='tanh', return_sequences=True))(embedding_layer)
context_vector = Attention(8)(bilstm)
simplernn = SimpleRNN(4, activation="tanh")(context_vector)
output = Dense(1, activation="sigmoid")(simplernn)

model_lstm_bi_embed_attention = Model(inputs=inputs, outputs=output)


In [ ]:
model_lstm_bi_embed_attention.summary()


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_20 (Embedding)    (None, 100, 200)          7825600   
                                                                 
 bidirectional_37 (Bidirect  (None, 100, 8)            6560      
 ional)                                                          
                                                                 
 attention_20 (Attention)    (None, 100, 8)            81        
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, 4)                 52        
                                                                 
 dense_57 (Dense)            (None, 1)                 5         
                                                           

In [ ]:
model_lstm_bi_embed_attention.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_model_lstm_bi_embed_attention = model_lstm_bi_embed_attention.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_dev, y_dev),verbose=1)

loss, accuracy = model_lstm_bi_embed_attention.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
157/157 [==============================] - 17s 77ms/step - loss: 0.6418 - accuracy: 0.6779 - val_loss: 0.5381 - val_accuracy: 0.8280
Epoch 2/10
157/157 [==============================] - 11s 70ms/step - loss: 0.5023 - accuracy: 0.8053 - val_loss: 0.4550 - val_accuracy: 0.8300
Epoch 3/10
157/157 [==============================] - 11s 69ms/step - loss: 0.4297 - accuracy: 0.8401 - val_loss: 0.4057 - val_accuracy: 0.8500
Epoch 4/10
157/157 [==============================] - 11s 71ms/step - loss: 0.3941 - accuracy: 0.8524 - val_loss: 0.4023 - val_accuracy: 0.8520
Epoch 5/10
157/157 [==============================] - 11s 69ms/step - loss: 0.3690 - accuracy: 0.8617 - val_loss: 0.3772 - val_accuracy: 0.8540
Epoch 6/10
157/157 [==============================] - 11s 70ms/step - loss: 0.3457 - accuracy: 0.8738 - val_loss: 0.3716 - val_accuracy: 0.8580
Epoch 7/10
157/157 [==============================] - 11s 70ms/step - loss: 0.3306 - accuracy: 0.8776 - val_loss: 0.3568 - val_accuracy:

<a name='5'></a>
## 5 - Model 2 - Attention Model with increased model complexity

In [ ]:
inputs = Input(shape=(100,))

embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(64, activation='tanh', return_sequences=True))(embedding_layer)
bilstm = Bidirectional(LSTM(128, activation='tanh',return_sequences=True))(bilstm)
context_vector = Attention(64)(bilstm)
simplernn = SimpleRNN(64, activation="tanh",return_sequences=True)(context_vector)

flatten = Flatten()(simplernn)

ffn = Dense(64, activation='relu')(flatten)
ffn = Dense(32, activation='relu')(ffn)

output = Dense(1, activation="sigmoid")(ffn)

model_lstm_bi_embed_attention_complex = Model(inputs=inputs, outputs=output)


In [ ]:
model_lstm_bi_embed_attention_complex.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_24 (Embedding)    (None, 100, 200)          7825600   
                                                                 
 bidirectional_44 (Bidirect  (None, 100, 128)          135680    
 ional)                                                          
                                                                 
 bidirectional_45 (Bidirect  (None, 100, 256)          263168    
 ional)                                                          
                                                                 
 attention_24 (Attention)    (None, 100, 256)          16513     
                                                                 
 simple_rnn_23 (SimpleRNN)   (None, 100, 64)           205

In [ ]:
model_lstm_bi_embed_attention_complex.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_model_lstm_bi_embed_attention_complex = model_lstm_bi_embed_attention_complex.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_dev, y_dev),verbose=1)

loss, accuracy = model_lstm_bi_embed_attention_complex.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
157/157 [==============================] - 21s 87ms/step - loss: 0.4781 - accuracy: 0.7560 - val_loss: 0.4265 - val_accuracy: 0.8160
Epoch 2/10
157/157 [==============================] - 12s 77ms/step - loss: 0.3445 - accuracy: 0.8542 - val_loss: 0.4771 - val_accuracy: 0.8200
Epoch 3/10
157/157 [==============================] - 12s 76ms/step - loss: 0.2949 - accuracy: 0.8815 - val_loss: 0.3254 - val_accuracy: 0.8760
Epoch 4/10
157/157 [==============================] - 12s 76ms/step - loss: 0.2613 - accuracy: 0.8958 - val_loss: 0.3103 - val_accuracy: 0.8720
Epoch 5/10
157/157 [==============================] - 12s 76ms/step - loss: 0.2280 - accuracy: 0.9111 - val_loss: 0.3327 - val_accuracy: 0.8640
Epoch 6/10
157/157 [==============================] - 12s 76ms/step - loss: 0.1998 - accuracy: 0.9210 - val_loss: 0.3239 - val_accuracy: 0.8900
Epoch 7/10
157/157 [==============================] - 12s 73ms/step - loss: 0.1650 - accuracy: 0.9387 - val_loss: 0.3169 - val_accuracy:

<a name='6'></a>
## 6 - Model 3 - Attention Model with increased model complexity and L2 regularization

In [ ]:
from keras.regularizers import l2

inputs = Input(shape=(100,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(64, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(embedding_layer)  # Applying L2 regularization
bilstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(bilstm)  # Applying L2 regularization
context_vector = Attention(64)(bilstm)
simplernn = SimpleRNN(64, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.001))(context_vector)  # Applying L2 regularization

flatten = Flatten()(simplernn)

ffn = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(flatten)  # Applying L2 regularization
ffn = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(ffn)  # Applying L2 regularization

output = Dense(1, activation="sigmoid")(ffn)

model_lstm_bi_embed_attention_complex_regularized = Model(inputs=inputs, outputs=output)


In [ ]:
model_lstm_bi_embed_attention_complex.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_24 (Embedding)    (None, 100, 200)          7825600   
                                                                 
 bidirectional_44 (Bidirect  (None, 100, 128)          135680    
 ional)                                                          
                                                                 
 bidirectional_45 (Bidirect  (None, 100, 256)          263168    
 ional)                                                          
                                                                 
 attention_24 (Attention)    (None, 100, 256)          16513     
                                                                 
 simple_rnn_23 (SimpleRNN)   (None, 100, 64)           205

In [ ]:
model_lstm_bi_embed_attention_complex_regularized.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_model_lstm_bi_embed_attention_complex_regularized = model_lstm_bi_embed_attention_complex_regularized.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_dev, y_dev),verbose=1)

loss, accuracy = model_lstm_bi_embed_attention_complex_regularized.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
313/313 [==============================] - 34s 81ms/step - loss: 0.8522 - accuracy: 0.6793 - val_loss: 0.5799 - val_accuracy: 0.8080
Epoch 2/10
313/313 [==============================] - 24s 75ms/step - loss: 0.5054 - accuracy: 0.8136 - val_loss: 0.4421 - val_accuracy: 0.8480
Epoch 3/10
313/313 [==============================] - 24s 76ms/step - loss: 0.4426 - accuracy: 0.8395 - val_loss: 0.4436 - val_accuracy: 0.8440
Epoch 4/10
313/313 [==============================] - 24s 75ms/step - loss: 0.3979 - accuracy: 0.8549 - val_loss: 0.3793 - val_accuracy: 0.8880
Epoch 5/10
313/313 [==============================] - 24s 75ms/step - loss: 0.3613 - accuracy: 0.8734 - val_loss: 0.3799 - val_accuracy: 0.8740
Epoch 6/10
313/313 [==============================] - 23s 75ms/step - loss: 0.3447 - accuracy: 0.8823 - val_loss: 0.4183 - val_accuracy: 0.8400
Epoch 7/10
313/313 [==============================] - 24s 75ms/step - loss: 0.3248 - accuracy: 0.8929 - val_loss: 0.3635 - val_accuracy:

<a name='7'></a>
## 7 - Model 4 - Hypertuning the model parameters

In [ ]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
import keras

import kerastuner as kt


def build_model(hp):
    inputs = Input(shape=(100,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
    bilstm = Bidirectional(LSTM(hp.Int('lstm_units', min_value=32, max_value=128, step=32), activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(embedding_layer)  # Applying L2 regularization
    bilstm = Bidirectional(LSTM(hp.Int('lstm_units', min_value=32, max_value=128, step=32), activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(bilstm)  # Applying L2 regularization
    context_vector = Attention(hp.Int('attention_units', min_value=32, max_value=128, step=32))(bilstm)
    simplernn = SimpleRNN(hp.Int('rnn_units', min_value=32, max_value=128, step=32), activation="tanh", return_sequences=True, kernel_regularizer=l2(0.001))(context_vector)  # Applying L2 regularization

    flatten = Flatten()(simplernn)

    ffn = Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu', kernel_regularizer=l2(0.01))(flatten)  # Applying L2 regularization
    ffn = Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu', kernel_regularizer=l2(0.01))(ffn)  # Applying L2 regularization

    output = Dense(1, activation="sigmoid")(ffn)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='./my_dir',  # Saving results to the current directory
                     project_name='hyperparameter_search')

tuner.search(X_train, y_train, epochs=10, validation_data=(X_dev, y_dev))

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_dev, y_dev))


Trial 30 Complete [00h 03m 58s]
val_accuracy: 0.515999972820282

Best val_accuracy So Far: 0.9039999842643738
Total elapsed time: 00h 52m 31s
Epoch 1/10
313/313 [==============================] - 31s 77ms/step - loss: 0.8653 - accuracy: 0.7216 - val_loss: 0.5203 - val_accuracy: 0.7840
Epoch 2/10
313/313 [==============================] - 22s 72ms/step - loss: 0.4649 - accuracy: 0.8291 - val_loss: 0.3962 - val_accuracy: 0.8560
Epoch 3/10
313/313 [==============================] - 22s 71ms/step - loss: 0.4399 - accuracy: 0.8362 - val_loss: 0.4352 - val_accuracy: 0.8360
Epoch 4/10
313/313 [==============================] - 22s 71ms/step - loss: 0.3949 - accuracy: 0.8627 - val_loss: 0.3611 - val_accuracy: 0.8660
Epoch 5/10
313/313 [==============================] - 23s 72ms/step - loss: 0.3590 - accuracy: 0.8775 - val_loss: 0.3384 - val_accuracy: 0.8960
Epoch 6/10
313/313 [==============================] - 22s 71ms/step - loss: 0.3348 - accuracy: 0.8880 - val_loss: 0.3715 - val_accuracy: 0

In [ ]:
best_hps.get_config()['values']

{'lstm_units': 64,
 'attention_units': 96,
 'rnn_units': 64,
 'dense_units': 128,
 'learning_rate': 0.001,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

<a name='8'></a>
## 8 - Model 4 - Attention Model Tuned with HyperParamerters on 10K dataset

In [ ]:
from keras.callbacks import EarlyStopping

lstm_units = 64
attention_units = 96
rnn_units = 64
dense_units = 128
learning_rate = 0.001

inputs = Input(shape=(100,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(embedding_layer)
bilstm = Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(bilstm)
context_vector = Attention(attention_units)(bilstm)
simplernn = SimpleRNN(rnn_units, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.001))(context_vector)
flatten = Flatten()(simplernn)
ffn = Dense(dense_units, activation='relu', kernel_regularizer=l2(0.01))(flatten)
ffn = Dense(dense_units, activation='relu', kernel_regularizer=l2(0.01))(ffn)
output = Dense(1, activation="sigmoid")(ffn)

model_lstm_bi_embed_attention_complex_regularized_tuned = Model(inputs=inputs, outputs=output)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate)
model_lstm_bi_embed_attention_complex_regularized_tuned.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_lstm_bi_embed_attention_complex_regularized_tuned.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 100, 200)          7825600   
                                                                 
 bidirectional_6 (Bidirecti  (None, 100, 128)          135680    
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 100, 128)          98816     
 onal)                                                           
                                                                 
 attention_2 (Attention)     (None, 100, 128)          12481     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 100, 64)           1235

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_model_lstm_bi_embed_attention_complex_regularized_tuned = model_lstm_bi_embed_attention_complex_regularized_tuned.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_dev, y_dev), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model_lstm_bi_embed_attention_complex_regularized_tuned.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/50
313/313 [==============================] - 23s 74ms/step - loss: 0.2935 - accuracy: 0.9049 - val_loss: 0.3507 - val_accuracy: 0.8920
Epoch 2/50
313/313 [==============================] - 23s 74ms/step - loss: 0.2742 - accuracy: 0.9147 - val_loss: 0.3456 - val_accuracy: 0.8960
Epoch 3/50
313/313 [==============================] - 23s 74ms/step - loss: 0.2555 - accuracy: 0.9252 - val_loss: 0.3871 - val_accuracy: 0.8720
Epoch 4/50
313/313 [==============================] - 23s 74ms/step - loss: 0.2467 - accuracy: 0.9279 - val_loss: 0.3462 - val_accuracy: 0.8900
Epoch 5/50
313/313 [==============================] - 23s 75ms/step - loss: 0.2249 - accuracy: 0.9405 - val_loss: 0.3699 - val_accuracy: 0.8860
Epoch 6/50
313/313 [==============================] - 23s 74ms/step - loss: 0.2010 - accuracy: 0.9500 - val_loss: 0.4522 - val_accuracy: 0.8820
Epoch 7/50
16/16 [==============================] - 0s 15ms/step - loss: 0.3897 - accuracy: 0.8660
Test Loss: 0.38966041803359985, Test 

In [ ]:
loss, accuracy = model_lstm_bi_embed_attention_complex_regularized_tuned.evaluate(X_dev, y_dev)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

16/16 [==============================] - 0s 21ms/step - loss: 0.3456 - accuracy: 0.8960
Test Loss: 0.345572829246521, Test Accuracy: 0.8960000276565552


<a name='9'></a>
## 9 - Model 5 - Attention Model Tuned with HyperParamerters on 1 Lakh dataset i.e.; 0.1 Million

In [ ]:
train_data = traindata.sample(n=100000, random_state=42)
test_data = testdata.sample(n=10000, random_state=42)

train_data['class_index'] = train_data['class_index'].map({1:0, 2:1})
test_data['class_index'] = test_data['class_index'].map({1:0, 2:1})

train_data['review_combined_lemma'] = train_data['review_combined_lemma'].fillna('')
test_data['review_combined_lemma'] = test_data['review_combined_lemma'].fillna('')

X_train = train_data.review_combined_lemma
y_train = np.array(train_data.class_index)

X = test_data.review_combined_lemma
y = np.array(test_data.class_index)

tokenizer = Tokenizer(oov_token="<UNK>",)
tokenizer.fit_on_texts(X_train)

tokenizer.word_index['<PAD>'] = 0

X_sequences_train = tokenizer.texts_to_sequences(X_train)

X_sequences = tokenizer.texts_to_sequences(X)

X_train = pad_sequences(X_sequences_train, maxlen=100)
X = pad_sequences(X_sequences, maxlen=100)

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

(100000, 100)
(5000, 100)
(5000, 100)


In [ ]:
vocab_size = len(tokenizer.word_index)
word2idx = tokenizer.word_index

embedding_matrix_twitter_200d = create_embedding_matrix(embedding_index_Twitter_200d, word2idx, vocab_size, 200)

In [ ]:

lstm_units = 64
attention_units = 96
rnn_units = 64
dense_units = 128
learning_rate = 0.001

inputs = Input(shape=(100,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=200, input_length=100, weights=[embedding_matrix_twitter_200d], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(embedding_layer)
bilstm = Bidirectional(LSTM(lstm_units, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.0001)))(bilstm)
context_vector = Attention(attention_units)(bilstm)
simplernn = SimpleRNN(rnn_units, activation="tanh", return_sequences=True, kernel_regularizer=l2(0.0001))(context_vector)
flatten = Flatten()(simplernn)
ffn = Dense(dense_units, activation='relu', kernel_regularizer=l2(0.001))(flatten)
ffn = Dense(dense_units, activation='relu', kernel_regularizer=l2(0.001))(ffn)
output = Dense(1, activation="sigmoid")(ffn)

model_lstm_bi_embed_attention_complex_regularized_tuned = Model(inputs=inputs, outputs=output)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate)
model_lstm_bi_embed_attention_complex_regularized_tuned.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_lstm_bi_embed_attention_complex_regularized_tuned.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 100, 200)          36072400  
                                                                 
 bidirectional_10 (Bidirect  (None, 100, 128)          135680    
 ional)                                                          
                                                                 
 bidirectional_11 (Bidirect  (None, 100, 128)          98816     
 ional)                                                          
                                                                 
 attention_4 (Attention)     (None, 100, 128)          12481     
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 100, 64)           1235

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_model_lstm_bi_embed_attention_complex_regularized_tuned = model_lstm_bi_embed_attention_complex_regularized_tuned.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_dev, y_dev), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model_lstm_bi_embed_attention_complex_regularized_tuned.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/50
3125/3125 [==============================] - 220s 68ms/step - loss: 0.3667 - accuracy: 0.8648 - val_loss: 0.3111 - val_accuracy: 0.8830
Epoch 2/50
3125/3125 [==============================] - 213s 68ms/step - loss: 0.2764 - accuracy: 0.9005 - val_loss: 0.2824 - val_accuracy: 0.8960
Epoch 3/50
3125/3125 [==============================] - 211s 68ms/step - loss: 0.2553 - accuracy: 0.9099 - val_loss: 0.2676 - val_accuracy: 0.9018
Epoch 4/50
3125/3125 [==============================] - 209s 67ms/step - loss: 0.2438 - accuracy: 0.9151 - val_loss: 0.2733 - val_accuracy: 0.8984
Epoch 5/50
3125/3125 [==============================] - 209s 67ms/step - loss: 0.2390 - accuracy: 0.9198 - val_loss: 0.7372 - val_accuracy: 0.4946
Epoch 6/50
3125/3125 [==============================] - 207s 66ms/step - loss: 0.2892 - accuracy: 0.9109 - val_loss: 0.2930 - val_accuracy: 0.8944
Epoch 7/50
3125/3125 [==============================] - 208s 67ms/step - loss: 0.2623 - accuracy: 0.9174 - val_loss: 0

In [ ]:
import pickle

with open('model_lstm_bi_embed_attention_complex_regularized_tuned.pkl', 'wb') as f:
    pickle.dump(model_lstm_bi_embed_attention_complex_regularized_tuned, f)